**ELGAMAL**

In [ ]:
import numpy as np
import math, sympy, random

# Discrete logarithm table, primitive roots. If order of element a is required, ..
def discrete_logarithm(Zp, p, phi_n, a=None):
  dis_log, roots, order = [], [], None
  for g in Zp:
    row = []
    for x in range(1, phi_n+1):
      val = pow(int(g), int(x), p)
      row.append(val)
    for j in row:
      if j == 1:
        if a:
          order = row.index(j)+1
        if row.index(j)+1 == phi_n:
          roots.append(g)
        break
    dis_log.append(row)
  return {"Discrete Logarithm": dis_log, 
          "Primitive roots": roots, 
          "order": order}


def multiplicative_group(Zp, p):
  mat = []
  for g in Zp:
    row = []
    for x in Zp:
      val = (g*x)%p
      if val not in Zp:
        return "Closure property not satisfied. Z "+ p + " is not a multiplicative group"
      row.append(val)
    if 1 not in row:
      return "Identity property not satisfied. Z "+ p + " is not a multiplicative group"
    mat.append(row)
  return mat

def generate_Zn(n):
  Zn = []
  for i in range(1, n):
    if math.gcd(i, n) == 1:
      Zn.append(i)
  return Zn

def euclideanModInverse(a, m):  
  if a == 0 :   
      return m, 0, 1
  gcd, x1, y1 = euclideanModInverse(m%a, a)  
  x = y1 - (m//a) * x1  
  y = x1  
  return gcd, x, y

def key_generation():
  while True:
    try:
      keys["Public Key"]["p"] = sympy.randprime(2, 1000)
      Zp, keys["d"] = generate_Zn(keys["Public Key"]["p"]), random.randint(1, keys["Public Key"]["p"]-2)
      di = discrete_logarithm(Zp, keys["Public Key"]["p"], len(Zp))
      keys["Public Key"]["e1"] = di["Primitive roots"][0]
      break
    except:
      continue
  keys["Public Key"]["e2"] = pow(keys["Public Key"]["e1"], keys["d"], keys["Public Key"]["p"])
  return Zp

def encryption(Zp, P):
  r = random.choice(Zp)
  return pow(keys["Public Key"]["e1"], r, keys["Public Key"]["p"]), (P*pow(keys["Public Key"]["e2"], r, keys["Public Key"]["p"]))%keys["Public Key"]["p"]

def decrypt(c1, c2):
  _, inv, _ = euclideanModInverse(pow(c1, keys["d"], keys["Public Key"]["p"]), keys["Public Key"]["p"])
  return (c2*inv)%keys["Public Key"]["p"]

def low_modulus_attack():
  e1, e2, p = keys["Public Key"]["e1"], keys["Public Key"]["e2"], keys["Public Key"]["p"]
  for i in range(1, p-1):
    if e2 == pow(e1, i, p):
      d = i
  return d

keys = {
    "Public Key": {
        "e1": None,
        "e2": None,
        "p": None
    },
    "d": None
}

Zp = key_generation()
P = random.randint(2, keys["Public Key"]["p"]-1)
c1, c2 = encryption(Zp, P)
print("Original number: "+str(P))
print("\nEncrypted number: "+str(c1)+ " , "+str(c2))
print("\nDecrypted number: "+str(decrypt(c1, c2)))
print("\nPrivate key using low modulus attack: " + str(low_modulus_attack()))


Original number: 90

Encrypted number: 152 , 35

Decrypted number: 90

Private key using low modulus attack: 155


**ELLIPTIC**

In [ ]:
def ECC_check_singular(a,b):
  return 4*a**3+27*b**2==0

def euclideanModInverse(a, m):  
    if a == 0 :   
        return m, 0, 1
    gcd, x1, y1 = euclideanModInverse(m%a, a)  
    x = y1 - (m//a) * x1  
    y = x1  
    return gcd, x, y

def ECC_point_addition(pt1, pt2, p):
  if pt1 == 'Pinf':
    return pt2
  if pt2 == 'Pinf':
    return pt1
  
  den = (pt2[0]-pt1[0])%p
  if den == 0:
    return 'Pinf'

  gcd, x, y = euclideanModInverse(den, p)
  ld = (pt2[1]-pt1[1])*x
  ld = ld%p
  x3 = (ld**2-pt1[0]-pt2[0])%p
  y3 = (ld*(pt1[0]-x3)-pt1[1])%p

  return [x3,y3]

def ECC_point_doubling(pt, p, a):
  if pt == 'Pinf':
    return pt
  den = 2*pt[1]%p
  if den == 0:
    return 'Pinf'

  gcd, x, y = euclideanModInverse(den, p)
  ld = (3*pt[0]**2+a)*x
  x3 = (ld**2-2*pt[0])%p
  y3 = (ld*(pt[0]-x3)-pt[1])%p
  return [x3,y3]

def ECC_point_inverse(pt, p):
  return 'Pinf' if pt=='Pinf' else [pt[0],-pt[1]%p]

def ECC_check_closure(table, points):
  for i in table:
    for j in i:
      if j not in points:
        return False
  return True

def ECC_check_inverse(points, p):
  for i in points:
    if ECC_point_inverse(i, p) not in points:
      return False
  return True

def key_generation(a, p):
  e1, d = [1, 9], 5 # e1 to be chosen from the group
  for i in range(d-1):
    e2 = ECC_point_doubling(e1, p, a) if i==0 else ECC_point_addition(e1, e2, p)
  return e1, e2, d


def encrypt(P, e1, e2, r, a, p):
  for i in range(r-1):
    c1 = ECC_point_doubling(e1, p, a) if i==0 else ECC_point_addition(e1, c1, p)
    c2 = ECC_point_doubling(e2, p, a) if i==0 else ECC_point_addition(e2, c2, p)
  c2 = ECC_point_addition(P, c2, p)
  return c1, c2

def decrypt(c1, c2, d, p):
  for i in range(d-1):
    P = ECC_point_doubling(c1, p, a) if i==0 else ECC_point_addition(c1, P, p)
  P = ECC_point_addition(ECC_point_inverse(P, p), c2, p)
  return P

def attack_using_r(c1, c2, e1, e2, a, p):
  temp, r = ECC_point_doubling(e1, p, a), 2
  while temp != c1:
    temp = ECC_point_addition(e1, temp, p)
    r += 1
  for i in range(r-1):
    P = ECC_point_doubling(e2, p, a) if i==0 else ECC_point_addition(e2, P, p)
  P = ECC_point_addition(ECC_point_inverse(P, p), c2, p)
  return P

def attack_using_d(c1, c2, e1, e2, a, p):
  temp, d = ECC_point_doubling(e1, p, a), 2
  while temp != e2:
    temp = ECC_point_addition(e1, temp, p)
    d += 1
  P = decrypt(c1, c2, d, p)
  return P

p, a, b = 13, 1, 1

points = [[i,j] for i in range(p) for j in range(p) if j**2%p == (i**3+a*i+b)%p]
points.insert(0, 'Pinf')

# table = [[ECC_point_addition(i, j, p) if i!=j else ECC_point_doubling(j, p, a) for j in points] for i in points]

e1, e2, d = key_generation(a, p)
pt = [10, 6]
c1, c2 = encrypt(pt, e1, e2, 7, a, p)

print("Encrypted points for the point "+ str(pt) +" is " + str(c1) + ", " + str(c2))
print("\n Attack using r: ")
print(attack_using_r(c1, c2, e1, e2, a, p))
print("\n Attack using d: ")
print(attack_using_d(c1, c2, e1, e2, a, p))


Encrypted points for the point [10, 6] is [12, 5], [12, 8]

 Attack using r: 
[10, 6]

 Attack using d: 
[10, 6]
